<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "16g") \
    .config("spark.executor.cores", "5")\
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
#DATA_PATH = '/workspace/data/criteo'
DATA_PATH = '/workspace'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')
SUBMIT_PATH = os.path.join(DATA_PATH, 'submission.csv')

PREPROCESSING_PIPELINE_PATH = os.path.join(DATA_PATH, 'preprocessing_pipeline')

In [3]:
os.getcwd()

'/workspace/MLBD/sgd_logreg_nn/notebooks'

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [84]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

In [85]:
df = df.sample(False, 0.5)

In [86]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (

Keeping only first two categorical features for simplicity

In [87]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [88]:
df = df.fillna(0, subset=num_columns)

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

In [91]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler

In [92]:
index_columns = list(map((lambda col: col + '_index'), cat_columns))
vector_columns = list(map((lambda col: col + '_vector'), cat_columns))

stages = [StringIndexer(inputCol=col, outputCol=ind_col, handleInvalid='keep') 
                   for col, ind_col in zip(cat_columns, index_columns)]

stages.append(OneHotEncoderEstimator(inputCols=index_columns, outputCols=vector_columns))
stages.append(VectorAssembler(inputCols=num_columns + vector_columns, outputCol='features'))

preprocessing_pipeline = Pipeline(stages=stages)

### Fit and save pipeline

In [93]:
preprocessing_pipeline_model = preprocessing_pipeline.fit(df)
preprocessing_pipeline_model.write().overwrite().save(PREPROCESSING_PIPELINE_PATH)

### Load fitted pipeline

In [94]:
from pyspark.ml import PipelineModel

In [95]:
preprocessing_pipeline_model = PipelineModel.load(PREPROCESSING_PIPELINE_PATH)

### Transform dataset using pipeline

In [96]:
df = preprocessing_pipeline_model.transform(df)
df = df.select('id', 'features', F.col('_c0').alias('label')).cache()

### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [35]:
from pyspark.sql import Window

In [36]:
def split_by_col(df, split_col, parts_fractions):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    sums = [[0, 0]]
    for pf in parts_fractions:
        sums.append((sums[-1][1], sums[-1][1] + pf))
        
    split_window = Window().orderBy(split_col)
    df = df.withColumn('fraction', F.percent_rank().over(split_window))
    
    return [df.filter(low < df['fraction']).filter(high >= df['fraction']).drop('fraction') for low, high in sums[1:]]

In [37]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

In [38]:
N = df.count()

In [39]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.7999992359759289, 0.09999983628055618, 0.10000038201203557)

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [40]:
from pyspark.ml.classification import LogisticRegression

In [44]:
lr = LogisticRegression(maxIter=100, regParam=0.01, elasticNetParam=0.1)
lr_model = lr.fit(train_df)

In [45]:
lr_model.transform(val_df).first()

Row(id=455266578481, features=SparseVector(1944, {3: 3.0, 4: 20288.0, 5: 14.0, 6: 1.0, 7: 3.0, 8: 11.0, 10: 1.0, 12: 3.0, 13: 1.0, 1494: 1.0}), label=0, rawPrediction=DenseVector([1.3868, -1.3868]), probability=DenseVector([0.8001, 0.1999]), prediction=0.0)

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [43]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [53]:
def rocauc(model, df):
    predictions = lr_model.transform(df).rdd.map(lambda row: (float(row['probability'][1]), float(row['label'])))
    return BinaryClassificationMetrics(predictions).areaUnderROC

def logloss(model, df):
    predictions = lr_model.transform(df).rdd.map(lambda row: (float(row['probability'][1]), float(row['label'])))
    return predictions.map(lambda row: -row[1] * np.log(row[0]) - (1. - row[1]) * np.log(1. - row[0])).mean()

def ne(model, df):
    label_mean = df.rdd.map(lambda row: float(row['label'])).mean()
    ll = -label_mean * np.log(label_mean) - (1. - label_mean) * np.log(1. - label_mean)
    return logloss(model, df) / ll

In [50]:
rocauc(lr_model, val_df)

0.7026663778468325

In [54]:
ne(lr_model, val_df)

0.9203179953421851

In [55]:
rocauc(lr_model, test_df)

0.7032453900744162

In [56]:
ne(lr_model, test_df)

0.9196657658665146

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [97]:
train_df, val_df = split_by_col(df, 'id', [0.9, 0.1])

In [98]:
df_test = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)


df_test = df_test.fillna(0, subset=num_columns)
df_test = preprocessing_pipeline_model.transform(df_test)
df_test = df_test.select('id', 'features').cache()

In [100]:
import time

best_params = None
best_rocauc = 0.
best_model = None

lr = LogisticRegression(maxIter=20)

for regParam in [0., 0.01, 0.05, 0.1, 0.2]: 
    for elasticNetParam in [0., 0.01, 0.05, 0.1, 0.2, 0.3]:
        t = time.time()
        lr.setParams(regParam=regParam, elasticNetParam=elasticNetParam)
        lr_model = lr.fit(train_df)
        metric = rocauc(lr_model, val_df)
        print(regParam, elasticNetParam, metric, "time:", time.time() - t)
        
        if metric > best_rocauc:
            best_rocauc = metric
            best_params = (regParam, elasticNetParam)
            best_model = lr_model
print("--------------------------------")
print("Best:", best_params, best_rocauc)

0.0 0.0 0.7027812779655739 time: 149.21538066864014
0.0 0.01 0.7027812779655739 time: 144.66799235343933
0.0 0.05 0.7027812779655739 time: 148.87183690071106
0.0 0.1 0.7027812779655739 time: 146.8454830646515
0.0 0.2 0.7027812779655739 time: 149.15402388572693
0.0 0.3 0.7027812779655739 time: 149.83160066604614
0.01 0.0 0.7015180440909613 time: 141.76691102981567
0.01 0.01 0.7020444697803493 time: 141.69171380996704
0.01 0.05 0.7030019958711722 time: 148.0107831954956
0.01 0.1 0.7028727523239727 time: 147.66852521896362
0.01 0.2 0.7012065215271714 time: 173.71195220947266
0.01 0.3 0.699128910974797 time: 169.24596738815308
0.05 0.0 0.6997982688771679 time: 151.2134394645691
0.05 0.01 0.7011692582181586 time: 137.39351201057434
0.05 0.05 0.6985748385245453 time: 140.28346872329712
0.05 0.1 0.6942933079141528 time: 141.90267515182495
0.05 0.2 0.6877523426890874 time: 144.2478904724121
0.05 0.3 0.6834199102624059 time: 140.88623094558716
0.1 0.0 0.6993529450927682 time: 139.15235424041748

In [103]:
import pandas as pd

df_predictions = best_model.transform(df_test).select('id', 'probability').toPandas()
df_predictions['proba'] = df_predictions['probability'].map(lambda p: p[1] if not pd.isna(p[1]) else 0.)
df_predictions[['id', 'proba']].to_csv(SUBMIT_PATH, index=False)